In [57]:
from datetime import datetime
from packaging import version

import os

In [58]:
import tensorflow as tf
print("Tensorflow version: ",tf.__version__)

Tensorflow version:  2.4.1


In [59]:
device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError("GPU device not found")
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


In [60]:
import tensorflow_datasets as tfds

In [61]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split = ['train','test'],
    shuffle_files = True,
    as_supervised = True,
    with_info = True,
)

In [62]:
def normalize_img(image, label):
    """" Normalizes images: uint8 -> float32 """
    return tf.cast(image, tf.float32) / 255., label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(normalize_img,num_parallel_calls=AUTOTUNE).cache()
ds_train = ds_train.batch(128).prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img,num_parallel_calls=AUTOTUNE).cache()
ds_test = ds_test.batch(128).prefetch(AUTOTUNE)

In [63]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = tf.keras.optimizers.Adam(0.001),
    metrics = ["accuracy"],
)

In [64]:
logs = "kayi/"+datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir = logs,
    histogram_freq = 1,
    profile_batch = '500,520'
)

model.fit(ds_train,epochs=2,validation_data=ds_test,callbacks = [tboard_callback])

Epoch 1/2
469/469 [==============================] - 3s 6ms/step - loss: 0.6142 - accuracy: 0.8323 - val_loss: 0.1902 - val_accuracy: 0.9463
Epoch 2/2
469/469 [==============================] - 3s 7ms/step - loss: 0.1759 - accuracy: 0.9500 - val_loss: 0.1372 - val_accuracy: 0.9601
